In [6]:
from pipeline.dl_models import MLP, CombinedMLP
from pipeline.preprocessing import feature_mfcc, feature_bandpower_struct, remove_high_frequencies
from pipeline.dataloader import PhonocardiogramAudioDataset, PhonocardiogramByIDDatasetOnlyResult
from pipeline.utils import compose_feature_label, audio_random_windowing
import pandas as pd

from tqdm import tqdm
import numpy as np

from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import re, random
import os

import librosa

## Prepare data

### Get label dictionary

In [4]:
dataset_info = pd.read_csv('assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv')

outcome_mapping = {'Normal': 1, 'Abnormal': 0}
dataset_info['Mapped_Outcome'] = dataset_info['Outcome'].map(outcome_mapping)
y_dict = dict(zip(dataset_info['Patient ID'], dataset_info['Mapped_Outcome']))

print(y_dict)

{2530: 0, 9979: 0, 9983: 0, 13918: 0, 14241: 0, 14998: 0, 23625: 0, 24160: 0, 29045: 0, 29378: 0, 31737: 0, 33151: 0, 36327: 0, 38337: 0, 39043: 0, 39403: 0, 39456: 0, 40058: 0, 40798: 0, 40840: 0, 43852: 1, 44514: 0, 45843: 0, 46065: 0, 46532: 1, 46579: 0, 46778: 0, 47002: 0, 49558: 0, 49561: 0, 49562: 0, 49568: 1, 49572: 1, 49574: 0, 49577: 1, 49585: 0, 49595: 0, 49598: 1, 49607: 0, 49610: 0, 49618: 0, 49622: 0, 49627: 0, 49628: 0, 49630: 0, 49631: 1, 49638: 0, 49641: 0, 49653: 1, 49659: 0, 49661: 1, 49669: 0, 49678: 1, 49683: 0, 49687: 0, 49691: 0, 49704: 0, 49712: 0, 49719: 1, 49729: 0, 49735: 0, 49745: 0, 49748: 0, 49751: 0, 49754: 0, 49761: 0, 49776: 0, 49808: 1, 49821: 0, 49823: 0, 49824: 0, 49829: 0, 49832: 1, 49838: 0, 49839: 1, 49842: 0, 49850: 0, 49853: 1, 49854: 0, 49873: 0, 49876: 0, 49896: 1, 49897: 1, 49900: 0, 49930: 1, 49931: 1, 49946: 0, 49952: 1, 49959: 1, 49960: 1, 49963: 0, 49966: 0, 49968: 1, 49969: 1, 49970: 1, 49974: 1, 49978: 0, 49979: 1, 49980: 0, 49983: 1, 49

In [7]:
def filter_files_by_keywords_and_extension(folder_path, keywords, extension):
    filtered_files = []
    for filename in os.listdir(folder_path):
        if any(keyword in filename for keyword in keywords) and filename.endswith(extension):
            filtered_files.append(filename)
    return filtered_files

folder_path = 'assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data'
keywords = ['TV','AV','PV','MV']
extension = '.wav'

filtered_files = filter_files_by_keywords_and_extension(folder_path, keywords, extension)
filtered_files[:5]


['50336_MV.wav',
 '50048_TV.wav',
 '68347_TV.wav',
 '50619_TV.wav',
 '84960_MV.wav']

In [26]:
import matplotlib.pyplot as plt
import librosa.display
import librosa
import torch

# Load the audio file
def get_features(file_path):
    sound, sr = librosa.load(file_path, sr=None)

    # Normalize the audio
    sound = sound / np.max(np.abs(sound))


    window_length_sec = 5
    window_length_samples = window_length_sec * sr
    windows = []
    for start in range(0, len(sound), window_length_samples):
        end = start + window_length_samples
        if end > len(sound):
            break
        window = sound[start:end]
        windows.append(window)

    spectrograms = []
    for window in windows:
        S = librosa.stft(window, n_fft=2048, hop_length=512)
        S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        spectrograms.append(S_db)
    
    return spectrograms



In [32]:
file_path = 'assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/2530_AV.wav'
a = get_features(file_path)

In [28]:
import pandas as pd
import soundfile
import re, os
from tqdm import tqdm
import warnings
def load_data(filtered_files):
    X, y = [], []
    count = 0
    for file in tqdm(filtered_files):
        file_path = os.path.join(folder_path, file)
        features = get_features(file_path)
        file_number = int(re.match(r'^([^_]*)', file)[1])
        label = y_dict[file_number]

        for feature in features:
            X.append(feature)
            y.append(label)
        count += 1

    print()
    return np.array(X), np.array(y)

warnings.filterwarnings('ignore')
features, labels = load_data(filtered_files)

100%|██████████| 3159/3159 [00:10<00:00, 290.76it/s]


In [31]:
print("the number of data features: ", len(features))
print("the number of data labels: ", len(labels))

the number of data features:  12929
the number of data labels:  12929


## Training

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("X_train.shape", X_train.shape)
print('y_train.shape', y_train.shape)

X_train.shape (10343, 1025, 40)
y_train.shape (10343,)


## Model1

In [34]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()

# Add convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))

# Add max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.005)  # Specify learning rate within Adam optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])

# Reshape X_train for 1D CNN
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32)

model.summary()


ValueError: cannot reshape array of size 424063000 into shape (10343,1025,1)

In [18]:
# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - accuracy: 0.5335 - loss: 0.6865 - recall: 0.7928
Evaluation Results:
Test Loss: 0.6957356333732605
Test Accuracy: 0.5228758454322815


## Model2

In [19]:
from keras.layers import Dropout

# Define the model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=64, validation_split=0.2)

model.summary()


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5030 - loss: 3.1771 - recall: 0.5091 - val_accuracy: 0.5000 - val_loss: 0.7436 - val_recall: 1.0000
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5205 - loss: 0.7303 - recall: 0.5952 - val_accuracy: 0.5122 - val_loss: 0.6921 - val_recall: 0.9184
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5351 - loss: 0.7010 - recall: 0.5849 - val_accuracy: 0.5000 - val_loss: 0.6922 - val_recall: 1.0000
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5177 - loss: 0.6953 - recall: 0.7327 - val_accuracy: 0.5000 - val_loss: 0.6921 - val_recall: 1.0000
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5219 - loss: 0.6915 - recall: 0.7292 - val_accuracy: 0.5653 - val_loss: 0.6901 - val_recall: 0.2408
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5142 - loss: 0.6930 - recall: 0.5279 - val_accuracy: 0.5878 - val_loss: 0.6908 - val_recall: 0.7184
Epoch 7/50
31/31

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 40, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 38, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       155,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,725 (1.86 MB)

 Trainable params: 162,241 (633.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 324,484 (1.24 MB)

In [20]:
# Reshape X_test for evaluation
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Evaluate the model
evaluation_results = model.evaluate(X_test_reshaped, y_test)

# Extracting individual evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

# Printing evaluation results
print("Evaluation Results:")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.6282 - loss: 0.6685 - recall: 0.4104
Evaluation Results:
Test Loss: 0.6727418303489685
Test Accuracy: 0.6127451062202454
